TODO:
- need to install every thing on Windows
- Error with c++ building llama-cpp-python

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class LowLevelModule:
    name: str
    description: str

@dataclass
class HighLevelModule:
    name: str
    description: str
    sub_modules: List[LowLevelModule]

@dataclass
class SoftwareArchitecture:
    system_prompt = "You are a software architect"
    prompt = """
    Code base:
    {code_base}
    ----

    Analyze this code base and carefully write a description of
    the software architecture.
    """

    name: str
    description: str
    modules: List[HighLevelModule]

    def to_markdown(self) -> str:
        lines = [
            f"# {self.name}\n",
            f"\n{fill(self.description, width=60)}\n\n",
        ]
        for module in self.modules:
            lines.append(f"## {module.name}\n\n")
            lines.append(
                f"**Description:** {fill(module.description, width=60)}\n\n"
            )
            lines.append("### Sub-modules\n\n")
            for sub_module in module.sub_modules:
                lines.append(f"**{sub_module.name}**\n\n")
                lines.append(f"{fill(sub_module.description, width=60)}\n\n")
        return "".join(lines)

In [ ]:
def main():
    repo_url = "https://github.com/advanced-stack/llm-components"
    with tempfile.TemporaryDirectory() as temp_dir:
        clone_dir = Path(temp_dir) / "repo"
        clone_repository(repo_url, clone_dir)

In [ ]:
from llm_components.loaders.code_base import map_codebase_to_text

code_base = map_codebase_to_text(clone_dir)

In [ ]:
with OpenAIAssistant(SoftwareArchitecture, model='gpt-4o') as assistant:
    software_architecture = assistant.process(code_base=code_base)
    markdown_output = software_architecture.to_markdown()
    print(markdown_output)